In [ ]:
import sys
import os
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
%matplotlib inline

In [ ]:
# parameters
# casename = 'COREII_Global'
casename = 'JRA55-do_Global'
var = 'mld_deltaR_mean'
# var = 'PE_delta'
# var = 'SST_mean'
diag_type = 'R'
# diag_type = 'D'

In [ ]:
# diagnostics
if diag_type == 'D':
    y_ref = 0
    y_label = '$\Delta$'
    if var == 'mld_deltaR_mean':
        ymin = -20
        ymax = 60
    elif var == 'PE_delta':
        ymin = -2e4
        ymax = 2e4
    elif var == 'SST_mean':
        ymin = -0.6
        ymax = 0.3
    else:
        raise ValueError('Variable {} not supported. Stop.'.format(var))
elif diag_type == 'R':
    y_ref = 1
    y_label = '$R$'
    if var == 'mld_deltaR_mean':
        ymin = 0.7
        ymax = 1.5
    elif var == 'PE_delta':
        ymin = 0.6
        ymax = 1.6
    elif var == 'SST_mean':
        ymin = 0.9
        ymax = 1.1
    else:
        raise ValueError('Variable {} not supported. Stop.'.format(var))
else:
    raise ValueError('Diagnostics type {} not supported. Stop.'.format(diag_type))

# prefix of figure
fig_prefix = 'fig_stat_diag_'+diag_type

In [ ]:
# paths
s2data_root = os.environ['GOTMFIG_ROOT']+'/data/'+casename
fig_root = os.environ['GOTMFIG_ROOT']+'/'+casename
os.makedirs(s2data_root, exist_ok=True)
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# lists
timetag_list = ['20090101-20090131',
                '20090201-20090228',
                '20090301-20090331',
                '20090401-20090430',
                '20090501-20090531',
                '20080601-20080630',
                '20080701-20080731',
                '20080801-20080831',
                '20080901-20080930',
                '20081001-20081031',
                '20081101-20081130',
                '20081201-20081231']
ntag = len(timetag_list)
turbmethod_list = ['KPP-CVMix',
                   'KPP-ROMS',
                   'EPBL',
                   'SMC',
                   'K-EPSILON-SG',
                   'KPPLT-EFACTOR',
                   'KPPLT-ENTR',
                   'KPPLT-RWHGK',
                   'EPBL-LT',
                   'SMCLT',
                   'OSMOSIS']
legend_list = ['KPP-CVMix',
               'KPP-ROMS',
               'ePBL',
               'SMC-KC94',
               'k-epsilon',
               'KPPLT-VR12',
               'KPPLT-LF17',
               'KPPLT-RWHGK16',
               'ePBL-LT',
               'SMCLT-H15',
               'OSMOSIS']
nm = len(turbmethod_list)
bcolor = ['lightcyan','lightblue','lightgreen','lightpink','palegoldenrod',
          'turquoise', 'deepskyblue','royalblue','forestgreen','firebrick','mediumpurple']

In [ ]:
# get data and save in a two dimensional list
gmobj_data_arr = []
gmobj_freg_mon = []
for j in np.arange(ntag):
    timetag = timetag_list[j]
    # read forcing regime
    s2data_name = s2data_root+'/VR1m_DT600s_'+timetag+'/data_forcing_regime_BG12_KPP-CVMix.npz'
    gmobj_tmp = GOTMMap().load(s2data_name)
    gmobj_freg_mon.append(gmobj_tmp)
    # read data
    gmobj_data_tmp = []
    for i in np.arange(nm):
        tmname = turbmethod_list[i]
        s2data_name = s2data_root+'/VR1m_DT600s_'+timetag+'/data_'+var+'_'+tmname+'.npz'
        gmobj_tmp = GOTMMap().load(s2data_name)
        gmobj_data_tmp.append(gmobj_tmp)
    gmobj_data_arr.append(gmobj_data_tmp)

In [ ]:
# median or mean (over all non-Langmuir turbulent methods) of var at each month, [12]*GOTMMap
gmobj_m_mon = []
# ratio of var for each turbulent methods over the median, [12][11]*GOTMMap
gmobj_stat_arr = []
# loop over time tags
for j in np.arange(ntag):
    ncase = gmobj_data_arr[j][0].data.size
    tmp = np.zeros([nm, ncase])
    # loop over turbulent methods
    for i in np.arange(nm):
        tmp[i,:] = gmobj_data_arr[j][i].data
#     np.ma.masked_invalid(tmp)
    # median of all non-Langmuir cases
#     diag0 = np.median(tmp[0:5,:], axis=0)
    # mean of all non-Langmuir cases
    diag0 = np.nanmean(tmp[0:5,:], axis=0)
    # get lon, lat etc.
    lon = gmobj_data_arr[j][0].lon
    lat = gmobj_data_arr[j][0].lat
    name = gmobj_data_arr[j][0].name
    units = gmobj_data_arr[j][0].units
    # create GOTMMap object
    gmobj_tmp = GOTMMap(data=diag0, lon=lon, lat=lat, name=name, units=units)
    # append to list
    gmobj_m_mon.append(gmobj_tmp)
    # tmp list
    gmobj_stat_tmp = []
    # loop over turbulent methods
    for i in np.arange(nm):
        if diag_type == 'D':
            # difference
            diag1 = tmp[i,:] - diag0
        elif diag_type == 'R':
            # ratio
            diag1 = tmp[i,:] / diag0
        # create GOTMMap object
        gmobj_tmp = GOTMMap(data=diag1, lon=lon, lat=lat, name=name, units=units)
        # append to list
        gmobj_stat_tmp.append(gmobj_tmp)
    # append to list
    gmobj_stat_arr.append(gmobj_stat_tmp)

In [ ]:
# figure 1: Statistics of the ratio to the median or mean
f = plt.figure()
f.set_size_inches(6, 4)

pbdata0 = []
xshift = list((np.arange(nm)-nm/2)*1)
for k in np.arange(nm):
    pdata = np.concatenate([gmobj_stat_arr[i][k].data for i in np.arange(12)])
    tmp = pdata
    pdata0 = pdata[~np.isnan(tmp)]
    pbdata0.append(pdata0)

# all the non-Langmuir cases
pdata1 = np.concatenate(pbdata0[0:5])
# all the Langmuir cases
pdata2 = np.concatenate(pbdata0[5:])
pbdata0.append(pdata1)
pbdata0.append(pdata2)
    
# add color for non-Lanmguir and Langmuir cases
pbcolor = bcolor
pbcolor.append('lightgray')
pbcolor.append('darkgray')
# add label for non-Lanmguir and Langmuir cases
label_list = legend_list
label_list.append('Non-Langmuir')
label_list.append('Langmuir')

# reference line
plt.axhline(y=y_ref, linewidth=1, color='gray')

# plot
xx = np.arange(nm+2)+1
position_arr = xx
pbox0 = plt.boxplot(pbdata0, whis=[5, 95], showfliers=False, positions=position_arr,
                         widths=0.3, patch_artist=True)
plt.ylim([ymin,ymax])
plt.xlim([0,nm+3])

# color for the  boxes
for patch, color in zip(pbox0['boxes'], pbcolor):
    patch.set_facecolor(color)
    
# x- and y-labels
ax = plt.gca()
plt.setp(ax, xticks=xx, xticklabels=label_list)
plt.ylabel(y_label)

# reduce margin
plt.tight_layout()

# auto adjust the x-axis label
plt.gcf().autofmt_xdate()

# save figure
figname = fig_root+'/'+fig_prefix+'_'+var+'_all.png'
plt.savefig(figname, dpi = 300)

In [ ]:
def mask_forcing_regime(pdata, pfreg):
    hist = np.zeros(14)
    pdata_S0= pdata[pfreg==1]
    pdata_S1 = pdata[pfreg==-1]
    hist[0] = pdata_S0.size
    hist[1] = pdata_S1.size
    pdata_L0= pdata[pfreg==2]
    pdata_L1 = pdata[pfreg==-2]
    hist[2] = pdata_L0.size
    hist[3] = pdata_L1.size
    pdata_C0= pdata[pfreg==3]
    pdata_C1 = pdata[pfreg==-3]
    hist[4] = pdata_C0.size
    hist[5] = pdata_C1.size
    pdata_SL0= pdata[pfreg==4]
    pdata_SL1 = pdata[pfreg==-4]
    hist[6] = pdata_SL0.size
    hist[7] = pdata_SL1.size
    pdata_SC0 = pdata[pfreg==5]
    pdata_SC1 = pdata[pfreg==-5]
    hist[8] = pdata_SC0.size
    hist[9] = pdata_SC1.size
    pdata_LC0 = pdata[pfreg==6]
    pdata_LC1 = pdata[pfreg==-6]
    hist[10] = pdata_LC0.size
    hist[11] = pdata_LC1.size
    pdata_SLC0 = pdata[pfreg==7]
    pdata_SLC1 = pdata[pfreg==-7]
    hist[12] = pdata_SLC0.size
    hist[13] = pdata_SLC1.size
    masked_data = [pdata_S0, pdata_S1, pdata_L0, pdata_L1, pdata_C0, pdata_C1, pdata_SL0, pdata_SL1,
               pdata_SC0, pdata_SC1, pdata_LC0, pdata_LC1, pdata_SLC0, pdata_SLC1]
    hist_pct = hist/np.sum(hist)*100
    return masked_data, hist_pct

In [ ]:
# figure 2: sort the differences by forcing regime

# forcing regimes
pfreg = np.concatenate([gmobj_freg_mon[i].data for i in np.arange(12)])

f, axarr = plt.subplots(2)
f.set_size_inches(8, 6)

# reference line
axarr[0].axhline(y=y_ref, linewidth=1, color='gray')
axarr[1].axhline(y=y_ref, linewidth=1, color='gray')

xshift = list((np.arange(nm)-nm/2)*0.07+0.035)
for k in np.arange(nm):
    pdata = np.concatenate([gmobj_stat_arr[i][k].data for i in np.arange(12)])
    tmp = pdata
    pfreg0 = pfreg[~np.isnan(tmp)]
    pdata0 = pdata[~np.isnan(tmp)]
    pbdata0, hist_pct = mask_forcing_regime(pdata0, pfreg0)
    xx = np.arange(hist_pct.size/2)+1
    position_arr = xx+xshift[k]
    pbox0 = axarr[0].boxplot(pbdata0[0::2], whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    pbox1 = axarr[1].boxplot(pbdata0[1::2], whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    for patch in pbox0['boxes']:
        patch.set_facecolor(bcolor[k])
    for patch in pbox1['boxes']:
        patch.set_facecolor(bcolor[k])

# x- and y-labels
plt.setp(axarr[0], xticks=xx,
         xticklabels=['S', 'L', 'C', 'SL', 'SC', 'LC', 'SLC'])
plt.setp(axarr[1], xticks=xx,
         xticklabels=['S*','L*','C*','SL*','SC*','LC*', 'SLC*'])
axarr[0].set_ylabel(y_label)
axarr[1].set_ylabel(y_label)
axarr[0].set_xlim([0.5,np.max(xx)+0.5])
axarr[1].set_xlim([0.5,np.max(xx)+0.5])
axarr[0].set_ylim([ymin,ymax])
axarr[1].set_ylim([ymin,ymax])

# frequency of occurrence
par1 = axarr[0].twinx()
par1.bar(xx, hist_pct[0::2], width=0.4, color='lightgray')
par1.set_ylabel('$\%$')
par1.set_ylim([0, 25])
axarr[0].set_zorder(par1.get_zorder()+1)
axarr[0].patch.set_visible(False)

par2 = axarr[1].twinx()
par2.bar(xx, hist_pct[1::2], width=0.4, color='lightgray')
par2.set_ylabel('$\%$')
par2.set_ylim([0, 25])
axarr[1].set_zorder(par2.get_zorder()+1)
axarr[1].patch.set_visible(False)

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/'+fig_prefix+'_'+var+'_forc_reg.png'
plt.savefig(figname, dpi = 300)
print(hist_pct)

In [ ]:
# figure 2b: sort the differences by forcing regime, only including L LC and C

# forcing regimes
pfreg = np.concatenate([gmobj_freg_mon[i].data for i in np.arange(12)])

f, axarr = plt.subplots(2)
f.set_size_inches(8, 6)

# reference line
axarr[0].axhline(y=y_ref, linewidth=1, color='gray')
axarr[1].axhline(y=y_ref, linewidth=1, color='gray')

xshift = list((np.arange(nm+2)-nm/2-1)*0.06+0.03)
pbdata0s0_nlt = []
pbdata0s1_nlt = []
pbdata0s0_lt = []
pbdata0s1_lt = []
for k in np.arange(nm):
    pdata = np.concatenate([gmobj_stat_arr[i][k].data for i in np.arange(12)])
    tmp = pdata
    pfreg0 = pfreg[~np.isnan(tmp)]
    pdata0 = pdata[~np.isnan(tmp)]
    pbdata0, hist_pct = mask_forcing_regime(pdata0, pfreg0)
    xx = np.arange(3)+1
    position_arr = xx+xshift[k]
    pbdata0s0 = [pbdata0[i] for i in [2,10,4]]
    pbox0 = axarr[0].boxplot(pbdata0s0, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.04, patch_artist=True)
    pbdata0s1 = [pbdata0[i] for i in [3,11,5]]
    pbox1 = axarr[1].boxplot(pbdata0s1, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.04, patch_artist=True)
    for patch in pbox0['boxes']:
        patch.set_facecolor(bcolor[k])
    for patch in pbox1['boxes']:
        patch.set_facecolor(bcolor[k])
    # save the data for non-Langmuir and Langmuir groups
    if k < 5:
        pbdata0s0_nlt.append(pbdata0s0)
        pbdata0s1_nlt.append(pbdata0s1)
    else:
        pbdata0s0_lt.append(pbdata0s0)
        pbdata0s1_lt.append(pbdata0s1)

# process data for non-Langmuir and Langmuir cases
pbdata1s0_nlt = []
pbdata1s1_nlt = []
pbdata1s0_lt = []
pbdata1s1_lt = []
for k in np.arange(3):
    pbdata0s0 = np.concatenate([pbdata0s0_nlt[i][k] for i in np.arange(len(pbdata0s0_nlt))])
    pbdata1s0_nlt.append(pbdata0s0)
    pbdata0s1 = np.concatenate([pbdata0s1_nlt[i][k] for i in np.arange(len(pbdata0s1_nlt))])
    pbdata1s1_nlt.append(pbdata0s1)
    pbdata0s0 = np.concatenate([pbdata0s0_lt[i][k] for i in np.arange(len(pbdata0s0_lt))])
    pbdata1s0_lt.append(pbdata0s0)
    pbdata0s1 = np.concatenate([pbdata0s1_lt[i][k] for i in np.arange(len(pbdata0s1_lt))])
    pbdata1s1_lt.append(pbdata0s1)
    
pbcolor = ['lightgray', 'darkgray']
# plot non-Langmuir cases
position_arr = xx+xshift[nm]
pbox0 = axarr[0].boxplot(pbdata1s0_nlt, whis=[5, 95], showfliers=False, positions=position_arr,
                         widths=0.04, patch_artist=True)
pbox1 = axarr[1].boxplot(pbdata1s1_nlt, whis=[5, 95], showfliers=False, positions=position_arr,
                         widths=0.04, patch_artist=True)
for patch in pbox0['boxes']:
    patch.set_facecolor(pbcolor[0])
for patch in pbox1['boxes']:
    patch.set_facecolor(pbcolor[0])
# plot Langmuir cases 
position_arr = xx+xshift[nm+1]
pbox0 = axarr[0].boxplot(pbdata1s0_lt, whis=[5, 95], showfliers=False, positions=position_arr,
                         widths=0.04, patch_artist=True)
pbox1 = axarr[1].boxplot(pbdata1s1_lt, whis=[5, 95], showfliers=False, positions=position_arr,
                         widths=0.04, patch_artist=True)
for patch in pbox0['boxes']:
    patch.set_facecolor(pbcolor[1])
for patch in pbox1['boxes']:
    patch.set_facecolor(pbcolor[1])

# x- and y-labels
plt.setp(axarr[0], xticks=xx, xticklabels=['L', 'LC', 'C'])
plt.setp(axarr[1], xticks=xx, xticklabels=['L*','LC*','C*'])
axarr[0].set_ylabel(y_label)
axarr[1].set_ylabel(y_label)
axarr[0].set_xlim([0.5,np.max(xx)+0.5])
axarr[1].set_xlim([0.5,np.max(xx)+0.5])
axarr[0].set_ylim([ymin,ymax])
axarr[1].set_ylim([ymin,ymax])

# frequency of occurrence
par1 = axarr[0].twinx()
hist_pcts1 = [hist_pct[i] for i in [2,10,4]]
par1.bar(xx, hist_pcts1, width=0.4, color='lightgray')
par1.set_ylabel('$\%$')
par1.set_ylim([0, 45])
axarr[0].set_zorder(par1.get_zorder()+1)
axarr[0].patch.set_visible(False)

par2 = axarr[1].twinx()
hist_pcts2 = [hist_pct[i] for i in [3,11,5]]
par2.bar(xx, hist_pcts2, width=0.4, color='lightgray')
par2.set_ylabel('$\%$')
par2.set_ylim([0, 45])
axarr[1].set_zorder(par2.get_zorder()+1)
axarr[1].patch.set_visible(False)

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/'+fig_prefix+'_'+var+'_forc_reg_s.png'
plt.savefig(figname, dpi = 300)
print(hist_pcts1)
print(hist_pcts2)
print(sum(hist_pcts1)+sum(hist_pcts2))

In [ ]:
def mask_latitude(pdata, lat):
    hist = np.zeros(7)
    pdata_S3 = pdata[lat<=-50]
    hist[0]  = pdata_S3.size
    pdata_S2 = pdata[(lat>-50) & (lat<=-30)]
    hist[1]  = pdata_S2.size
    pdata_S1 = pdata[(lat>-30) & (lat<=-10)]
    hist[2]  = pdata_S1.size
    pdata_EQ = pdata[(lat>-10) & (lat<10)]
    hist[3]  = pdata_EQ.size
    pdata_N1 = pdata[(lat>=10) & (lat<30)]
    hist[4]  = pdata_N1.size
    pdata_N2 = pdata[(lat>=30) & (lat<50)]
    hist[5]  = pdata_N2.size
    pdata_N3 = pdata[lat>=50]
    hist[6]  = pdata_N3.size
    masked_data = [pdata_S3, pdata_S2, pdata_S1, pdata_EQ, pdata_N1, pdata_N2, pdata_N3]
    hist_pct = hist/np.sum(hist)*100
    return masked_data, hist_pct

In [ ]:
# figure 3: sort the differences by latitude

plat = np.concatenate([gmobj_m_mon[i].lat for i in np.arange(12)])

f = plt.figure()
f.set_size_inches(8, 3.5)

# reference line
plt.axhline(y=y_ref, linewidth=1, color='gray')

xshift = list((np.arange(nm)-nm/2)*0.07+0.035)
for k in np.arange(nm):
    pdata = np.concatenate([gmobj_stat_arr[i][k].data for i in np.arange(12)])
    tmp = pdata
    plat0 = plat[~np.isnan(tmp)]
    pdata0 = pdata[~np.isnan(tmp)]
    pbdata0, hist_pct = mask_latitude(pdata0, plat0)
    xx = np.arange(hist_pct.size)+1
    position_arr = xx+xshift[k]
    pbox0 = plt.boxplot(pbdata0, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    for patch in pbox0['boxes']:
        patch.set_facecolor(bcolor[k])

# x- and y-labels
ax = plt.gca()
plt.setp(ax, xticks=xx, xticklabels=['70$^\circ$S-50$^\circ$S',
                                     '50$^\circ$S-30$^\circ$S',
                                     '30$^\circ$S-10$^\circ$S',
                                     '10$^\circ$S-10$^\circ$N',
                                     '10$^\circ$N-30$^\circ$N',
                                     '30$^\circ$N-50$^\circ$N',
                                     '50$^\circ$N-70$^\circ$N'])
plt.ylabel(y_label)
plt.xlim([0.5,np.max(xx)+0.5])
plt.ylim([ymin,ymax])

# frequency of occurrence
par1 = ax.twinx()
par1.bar(xx, hist_pct, width=0.4, color='lightgray')
par1.set_ylabel('$\%$')
par1.set_ylim([0, 25])
ax.set_zorder(par1.get_zorder()+1)
ax.patch.set_visible(False)

# reduce margin
plt.tight_layout()

# auto adjust the x-axis label
plt.gcf().autofmt_xdate()

# save figure
figname = fig_root+'/'+fig_prefix+'_'+var+'_lat.png'
plt.savefig(figname, dpi = 300)

In [ ]:
# figure 4: sort the differences by month

f, axarr = plt.subplots(3)
f.set_size_inches(12, 9)

# reference line
axarr[0].axhline(y=y_ref, linewidth=1, color='gray')
axarr[1].axhline(y=y_ref, linewidth=1, color='gray')
axarr[2].axhline(y=y_ref, linewidth=1, color='gray')

xx = np.arange(12)+1
xshift = list((np.arange(nm)-nm/2)*0.07+0.035)
for k in np.arange(nm):
    pbdata0 = []
    pbdata1 = []
    pbdata2 = []
    for i in np.arange(12):
        plat = gmobj_stat_arr[i][k].lat
        # south of 30S
        pdata = gmobj_stat_arr[i][k].data[plat<=-30]
        pdata = pdata[~np.isnan(pdata)]
        pbdata0.append(pdata)
        # 30S-30N
        pdata = gmobj_stat_arr[i][k].data[(plat>-30) & (plat<30)]
        pdata = pdata[~np.isnan(pdata)]
        pbdata1.append(pdata)
        # north of 30N
        pdata = gmobj_stat_arr[i][k].data[plat>=30]
        pdata = pdata[~np.isnan(pdata)]
        pbdata2.append(pdata)
    position_arr = xx+xshift[k]
    pbox0 = axarr[0].boxplot(pbdata0, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    pbox1 = axarr[1].boxplot(pbdata1, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    pbox2 = axarr[2].boxplot(pbdata2, whis=[5, 95], showfliers=False, positions=position_arr,
                             widths=0.05, patch_artist=True)
    for patch in pbox0['boxes']:
        patch.set_facecolor(bcolor[k])
    for patch in pbox1['boxes']:
        patch.set_facecolor(bcolor[k])
    for patch in pbox2['boxes']:
        patch.set_facecolor(bcolor[k])

# x- and y-labels
month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
plt.setp(axarr[0], xticks=xx, xticklabels=month_labels)
plt.setp(axarr[1], xticks=xx, xticklabels=month_labels)
plt.setp(axarr[2], xticks=xx, xticklabels=month_labels)
axarr[0].set_ylabel(y_label)
axarr[1].set_ylabel(y_label)
axarr[2].set_ylabel(y_label)
axarr[0].set_xlim([0.5,np.max(xx)+0.5])
axarr[1].set_xlim([0.5,np.max(xx)+0.5])
axarr[2].set_xlim([0.5,np.max(xx)+0.5])
axarr[0].set_ylim([ymin,ymax])
axarr[1].set_ylim([ymin,ymax])
axarr[2].set_ylim([ymin,ymax])

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/'+fig_prefix+'_'+var+'_mon.png'
plt.savefig(figname, dpi = 300)